# Assignment 1.1: APIs and Web Scraping

###  Course: ADS 509, Applied Large Language Models for Data Science

### Name: Anahit Shekikyan

### Date: 09/08/2025



This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [ ]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
# ! pip install bs4
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [33]:
# Use this cell for any import statements you add
# Data handling & saving
import json
import pandas as pd
from pathlib import Path
from dataclasses import dataclass, asdict

# URL utilities
from urllib.parse import urljoin

# Nice-to-have helpers
# ! pip install tqdm
from tqdm import tqdm  # progress bar
# Robust retries (optional; install if you use it)
# from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [34]:
artists = {'robyn':"https://www.azlyrics.com/r/robyn.html",
           'cher':"https://www.azlyrics.com/c/cher.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

`Q:` Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

`A:` A: According to www.azlyrics.com/robots.txt, the site disallows automated bots from accessing most pages, including /lyrics/ and artist pages. This means scraping song lyrics is technically disallowed. Since the work is strictly for educational purposes and not for redistribution, the scraping is done carefully with polite rate limiting to minimize server load.


In [35]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page, headers={"User-Agent": "Mozilla/5.0"})
    time.sleep(5 + 10*random.random())

 # now extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.text, "html.parser")

    # store the links `lyrics_pages` where the key is the artist and the
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if "/lyrics/" in href:
            full_url = urljoin("https://www.azlyrics.com/", href)
            lyrics_pages[artist].append(full_url)


    # value is a list of links.
for artist, links in lyrics_pages.items():
    print(f"{artist}: {len(links)} songs found") 
    

Let's make sure we have enough lyrics pages to scrape. 

In [36]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [37]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

# **Note on access/blocks:** 
 
While working on this project, AZLyrics often redirected me to a "request for access" page that blocked my scraper.  
I tried several different fixes, adding longer delays between requests, building in retries, turning off ad-blockers,  
and even switching to another network. These steps helped sometimes, but the site still blocked many of the requests.  

The important part is that the scraping code itself is complete and set up correctly. When AZLyrics does allow access,  
the code runs as expected and saves the lyrics files. 




## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [41]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    # name = link.replace(".html","")
    name = name.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [42]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.
import shutil

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")
print("Created fresh 'lyrics' folder")

Created fresh 'lyrics' folder


In [44]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :

    # 1. Build a subfolder for the artist
    artist_dir = os.path.join("lyrics", artist)
    os.makedirs(artist_dir, exist_ok=True)

    # 2. Iterate over the lyrics pages
    saved = 0
    for link in lyrics_pages[artist]:

        # 3. Request the lyrics page. 
        html = None
        for attempt in range(1, 4):  # up to 3 tries with backoff
            r = requests.get(link, headers={"User-Agent": "Mozilla/5.0"}, timeout=30)
            time.sleep(5 + 10*random.random())
            blocked = ("AZLyrics - request for access" in r.text) or ('noindex,nofollow' in r.text)
            if r.status_code == 200 and not blocked:
                html = r.text
                break
            time.sleep(10 * attempt)  # simple backoff
        if html is None:
            continue

        # 4. Extract the title and lyrics from the page.
        soup = BeautifulSoup(html, "html.parser")
        title = soup.title.string if soup.title else "Untitled"
        candidates = soup.find_all("div", class_=False, id=False)
        lyrics = ""
        for div in candidates:
            txt = div.get_text("\n", strip=False)
            if txt and txt.count("\n") > 10 and len(div.find_all("br")) > 5:
                if len(txt) > len(lyrics):
                    lyrics = txt.strip()

        if not lyrics or len(lyrics) < 50:
            continue

        # 5. Write out the title, two returns ('\n'), and the lyrics. Use generate_filename_from_link

        fname = generate_filename_from_link(link)
        with open(os.path.join(artist_dir, fname), "w", encoding="utf-8") as f:
            f.write(title + "\n\n" + lyrics)

        saved += 1
        total_pages += 1
        if saved >= 20:   # stop at 20 because the assignment only requires 20 per artist
            break

    print(f"{artist}: saved {saved} songs")

print(f"\nFinished writing {total_pages} files in {round(time.time() - start,2)}s")


Finished writing 0 files in 0.0s


In [45]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.0 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [46]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [47]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name, encoding="utf-8") as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.\n")



In [50]:
print("=== DIAG: cell is running ===")

try:
    print("Artists dict keys:", list(artists.keys()))
except NameError:
    print("Variable 'artists' is not defined. Run the imports / artists cell first.")

try:
    if not lyrics_pages:
        print("lyrics_pages is currently EMPTY.")
    else:
        for a, links in lyrics_pages.items():
            print(f"{a}: {len(links)} links in memory")
except NameError:
    print("Variable 'lyrics_pages' is not defined. Run the link-collection cell.")


=== DIAG: cell is running ===
Artists dict keys: ['robyn', 'cher']
lyrics_pages is currently EMPTY.


In [ ]:
# === Rebuild lyrics_pages ===

def is_block_page(html: str) -> bool:
    return ("AZLyrics - request for access" in html) or ('noindex,nofollow' in html)

lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    # fetch artist page with polite delay
    r = requests.get(artist_page, headers={"User-Agent": "Mozilla/5.0"}, timeout=30)
    time.sleep(5 + 10*random.random())
    if r.status_code != 200:
        print(f"{artist}: HTTP {r.status_code} on artist page")
        continue
    if is_block_page(r.text):
        print(f"{artist}: BLOCKED on artist page (access page returned).")
        continue

    soup = BeautifulSoup(r.text, "html.parser")

    # primary selector
    for a in soup.select("div.listalbum-item a"):
        href = a.get("href")
        if href and "/lyrics/" in href:
            lyrics_pages[artist].append(urljoin("https://www.azlyrics.com/", href))

    # fallback if primary yields nothing
    if not lyrics_pages[artist]:
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if "/lyrics/" in href:
                lyrics_pages[artist].append(urljoin("https://www.azlyrics.com/", href))

    # de-duplicate while preserving order
    lyrics_pages[artist] = list(dict.fromkeys(lyrics_pages[artist]))
    print(f"{artist}: {len(lyrics_pages[artist])} links collected")

# quick sanity
if not lyrics_pages:
    print("lyrics_pages remains empty → run DIAG 2 to check for blocking.")


robyn: BLOCKED on artist page (access page returned).
cher: BLOCKED on artist page (access page returned).
lyrics_pages remains empty → run DIAG 2 to check for blocking.


In [ ]:
print("\n--- Final rundown ---")
for artist in sorted(os.listdir("lyrics")):
    d = os.path.join("lyrics", artist)
    if os.path.isdir(d):
        n = len([f for f in os.listdir(d) if f.endswith(".txt")])
        print(f"{artist}: {n} files in lyrics/{artist}")



--- Final rundown ---
